# Hash Maps/Dictionaries

## How They Work

Dictionaries map key:value pairs to specific locations in an array. A **hash function** takes a **key** and creates a **hash code/value**, which determines where to store the value in the array.

Pros:
- Fast Insertion
- Fast Deletion
- Fast Look-Up

Considerations:
- Collision Resolution: how to handle when two keys map to the same index
  - Linear Probing: Assigns key to the next available index (leads to clustering and reduces to O(n))
  - Double Hashing: Instead of going to the next available index, the hash2 depends on the key s.t. `SS = C - (key % C)` where C is a small prime number < len(HT). Here, the final hash would be `(hash(key) + i * SS(key)) % len(HT)` where i is incremented for every collision. Critically, len(HT) **must be prime** or the entire table will not be probable for some keys.
  - Separate Chaining: Each index in the array is a linked list, and new values are inserted at the head; maintains O(1) insertion but O(n/k) searching
- Good Hash Functions:
  - Minimize collisions
  - Make use of all info provided by key
  - Uniformly distributes output across **entire** table
  - Similar keys still map to different hash values
  - Uses only fast operations


Common Hash Functions:
- Numeric keys: `key % len(HT)`

Transforming lowercase keys to unique integers:
- Lowercase keys: `sum((ord(c) - 96) * 26 ** (len(key) - i - 1) for i, c in enumerate(key))`  
  'ace' -> `1 * 26 ** 2 + 3 * 26 ** 1 + 5 * 26 ** 0`  
  More efficient to compute as `((1) * 26 + 2) * 26 + 5`

## Sample Implementation

In [ ]:
from typing import Any

class HashMap:
    """Implements a static-length, double-hashing map that accepts lower-case string keys
    """

    def __init__(self, map_size: int, C: int) -> None:
        self._C = self._map_size = None
        self.map_size = map_size
        self.C = C
        self._HM = [(None, None, False) for _ in range(self._map_size)]
        self._empty_indices = self._map_size

    def _isPrime(self, n: int) -> bool:
        if (n < 2 or n % 2 == 0):
            return n == 2

        divisor = 3
        while (divisor <= n ** 0.5):
            if (n % divisor == 0):
                return False
            else:
                divisor += 2

        return True

    @property
    def map_size(self) -> int:
        return self._map_size

    @map_size.setter
    def map_size(self, map_size_val: int) -> None:
        if self._map_size is not None:
            raise RuntimeError("map_size cannot be modified after instantiation.")
        if not isinstance(map_size_val, int):
            raise TypeError("map_size is not an int.")
        if not (map_size_val > 3):
            raise ValueError("map_size must be greater than 3.")
        if not self._isPrime(map_size_val):
            raise ValueError("map_size must be a prime number")

        self._map_size = map_size_val

    @property
    def C(self) -> int:
        return self._C

    @C.setter
    def C(self, C_val: int) -> None:
        if self._C is not None:
            raise RuntimeError("C cannot be modified after instantiation.")
        if not isinstance(C_val, int):
            raise TypeError("C is not an int.")
        if not (2 < C_val < self._map_size):
            raise ValueError("C must be between 2 and map_size (exclusive)")
        if not self._isPrime(C_val):
            raise ValueError("C must be a prime number")

        self._C = C_val

    @property
    def _out_of_space(self) -> bool:
        if self._empty_indices == 0:
            return True
        else:
            return False
    
    def _lowercase_key_to_unique_int(self, key_str: str) -> int:
        key_int = 0

        for c in key_str:
            key_int = key_int * 26 + (ord(c) - 96)

        return key_int
    
    def _generate_hash1(self, key_int: int) -> int:
        return key_int % self._map_size
    
    def _generate_hash2(self, key_int: int) -> int:
        return (self._C - (key_int % self._C))

    def _generate_double_hash(self, hash1: int, hash2: int,  i: int) -> int:
        return (hash1 + i * hash2) % self._map_size

    def _get_hash_index(self, key: str, skip_modified: bool) -> int:
        key_int = self._lowercase_key_to_unique_int(key_str=key)
        hash1 = self._generate_hash1(key_int)
        hash2 = self._generate_hash2(key_int)

        for i in range(self._map_size):
            current_index = self._generate_double_hash(hash1, hash2, i)

            if self._key_at_index(current_index) == key:
                return current_index
            if self._key_at_index(current_index) is None:
                if not skip_modified:
                    return current_index
                elif not self._index_ever_modified(current_index):
                    return current_index

        return -1

    def _add_key_value_pair_at_index(self, key: str, value: Any, index: int) -> None:
        self._HM[index] = (key, value, True)
        self._empty_indices -= 1

    def _key_at_index(self, index: int) -> int:
        return self._HM[index][0]

    def _value_at_index(self, index: int) -> Any:
        return self._HM[index][1]

    def _index_ever_modified(self, index: int) -> bool:
        return self._HM[index][2]

    def _clear_index(self, index: int) -> None:
        self._HM[index] = (None, None, True)
        self._empty_indices += 1

    def clear(self) -> None:
        self._HM = [(None, None, False) for _ in range(self._map_size)]
        self._empty_indices = self._map_size

    def _validate_key(self, key: str) -> None:
        if not isinstance(key, str):
            raise TypeError("Key is not type str.")
        if not (key.islower() and key.isalpha()):
            raise ValueError("Key does not consist of only lowercase characters (a-z).")
    
    def add(self, key: str, value: Any) -> None:
        if self._out_of_space:
            raise MemoryError(f'Hash map of size {self._map_size} is full.')
        
        self._validate_key(key)

        index = self._get_hash_index(key, skip_modified=False)

        if self._key_at_index(index) == key:
            raise KeyError("Key is already in the hash map.")
        elif self._key_at_index(index) is None:
            self._add_key_value_pair_at_index(key, value, index)

    def retrieve(self, key: str) -> Any:
        self._validate_key(key)

        index = self._get_hash_index(key, skip_modified=True)

        if (index == -1) or (self._key_at_index(index) is None):
            raise KeyError("Key is not in hash map.")
        elif self._key_at_index(index) == key:
            return self._value_at_index(index)

    def update(self, key: str, new_value: Any) -> None:
        self._validate_key(key)

        index = self._get_hash_index(key, skip_modified=True)

        if (index == -1) or (self._key_at_index(index) is None):
            raise KeyError("Key is not in hash map.")
        elif self._key_at_index(index) == key:
            self._add_key_value_pair_at_index(key, new_value, index)

    def delete(self, key: str) -> None:
        self._validate_key(key)

        index = self._get_hash_index(key, skip_modified=True)

        if (index == -1) or (self._key_at_index(index) is None):
            raise KeyError("Key is not in hash map.")
        elif self._key_at_index(index) == key:
            self._clear_index(index)

    def pop(self, key: str) -> Any:
        self._validate_key(key)

        index = self._get_hash_index(key, skip_modified=True)

        if (index == -1) or (self._key_at_index(index) is None):
            raise KeyError("Key is not in hash map.")
        elif self._key_at_index(index) == key:
            value = self._value_at_index(index)
            self._clear_index(index)
            return value